In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('db_basic_queries.sqlite3')

In [6]:
query = """
SELECT * FROM pedidos
"""

pd.read_sql_query(query, conn)

,pedido_id,cliente_id,producto,cantidad,precio_unit,fecha_pedido,estado
0,101,1.0,Laptop,1,3500.0,2025-02-10,ENVIADO
1,102,1.0,Mouse,2,45.5,2025-02-10,ENTREGADO
2,103,2.0,Teclado,1,80.0,2025-03-25,PENDIENTE
3,104,3.0,Monitor,2,220.0,2024-12-01,CANCELADO
4,105,4.0,Smartphone,1,1200.0,2025-07-02,ENVIADO
5,106,5.0,Tablet,3,300.0,2025-06-20,ENTREGADO
6,107,2.0,Auriculares,1,65.0,2025-03-26,ENTREGADO
7,108,NaN,Webcam,1,70.0,2025-05-05,PENDIENTE


In [7]:
query = """
SELECT * FROM clientes
"""

pd.read_sql_query(query, conn)

,cliente_id,nombre,ciudad,edad,fecha_registro
0,1,Ana Gómez,Bogotá,34,2025-01-15
1,2,Luis Pérez,Medellín,28,2025-03-22
2,3,María Ruiz,Cali,41,2024-11-07
3,4,Juan Torres,None,22,2025-07-01
4,5,Carla León,Bogotá,29,2025-06-18


¿Qué pedidos con estado “ENVIADO” o “ENTREGADO” tienen más de una unidad y fueron realizados entre el 1 de enero y el 30 de junio de 2025?

In [8]:
query = """
SELECT * FROM pedidos WHERE estado IN ("ENVIADO", "ENTREGADO") AND fecha_pedido BETWEEN "2025-01-01" AND "2025-06-30"
"""
pd.read_sql_query(query, conn)

,pedido_id,cliente_id,producto,cantidad,precio_unit,fecha_pedido,estado
0,101,1,Laptop,1,3500.0,2025-02-10,ENVIADO
1,102,1,Mouse,2,45.5,2025-02-10,ENTREGADO
2,106,5,Tablet,3,300.0,2025-06-20,ENTREGADO
3,107,2,Auriculares,1,65.0,2025-03-26,ENTREGADO


¿Qué clientes han hecho más de un pedido, y cuál es el número de pedidos y el total gastado por cada uno?

In [9]:
query = """
SELECT clientes.cliente_id, clientes.nombre, COUNT(pedidos.cliente_id) AS numero_pedidos,
SUM(pedidos.precio_unit * pedidos.cantidad) AS total_gastado
FROM pedidos INNER JOIN clientes
ON clientes.cliente_id = pedidos.cliente_id
GROUP BY pedidos.cliente_id
HAVING COUNT(pedidos.cliente_id) > 1
"""
pd.read_sql_query(query, conn)

,cliente_id,nombre,numero_pedidos,total_gastado
0,1,Ana Gómez,2,3591.0
1,2,Luis Pérez,2,145.0


¿Cómo se clasifica el nivel de gasto de cada cliente en “ALTO”, “MEDIO” o “BAJO”, según el total que ha gastado?

In [10]:
query = """
SELECT clientes.cliente_id, clientes.nombre, SUM(pedidos.precio_unit * pedidos.cantidad) AS total_gastado,
CASE
    WHEN SUM(pedidos.precio_unit * pedidos.cantidad) >= 1000 THEN 'ALTO'
    WHEN SUM(pedidos.precio_unit * pedidos.cantidad) >= 500 AND SUM(pedidos.precio_unit * pedidos.cantidad) < 1000 THEN 'MEDIO'
    ELSE 'BAJO'
END AS clasificacion
FROM pedidos INNER JOIN clientes
ON clientes.cliente_id = pedidos.cliente_id
GROUP BY pedidos.cliente_id
"""
pd.read_sql_query(query, conn)

,cliente_id,nombre,total_gastado,clasificacion
0,1,Ana Gómez,3591.0,ALTO
1,2,Luis Pérez,145.0,BAJO
2,3,María Ruiz,440.0,BAJO
3,4,Juan Torres,1200.0,ALTO
4,5,Carla León,900.0,MEDIO


¿Cuáles son las ciudades únicas de los clientes (no nulas), ordenadas alfabéticamente? Muestre los resultados(filas) 2,3 y 4, es decir, muestre tres resultados a partir del segundo.

In [11]:
query = """
SELECT * FROM clientes
"""
pd.read_sql_query(query, conn)

,cliente_id,nombre,ciudad,edad,fecha_registro
0,1,Ana Gómez,Bogotá,34,2025-01-15
1,2,Luis Pérez,Medellín,28,2025-03-22
2,3,María Ruiz,Cali,41,2024-11-07
3,4,Juan Torres,None,22,2025-07-01
4,5,Carla León,Bogotá,29,2025-06-18


In [25]:
query = """
SELECT clientes.ciudad FROM clientes
WHERE clientes.ciudad IS NOT NULL
ORDER BY clientes.ciudad ASC
"""
pd.read_sql_query(query, conn)

,ciudad
0,Bogotá
1,Bogotá
2,Cali
3,Medellín


In [28]:
query = """
SELECT estado, SUM(cantidad), MIN(cantidad), MAX(cantidad), AVG(cantidad) FROM pedidos GROUP BY estado
"""
pd.read_sql_query(query, conn)

,estado,SUM(cantidad),MIN(cantidad),MAX(cantidad),AVG(cantidad)
0,CANCELADO,2,2,2,2.0
1,ENTREGADO,6,1,3,2.0
2,ENVIADO,2,1,1,1.0
3,PENDIENTE,2,1,1,1.0


**CTE**

In [5]:
query = """
WITH cantidad_clientes AS (
SELECT cliente_id, SUM(cantidad) as cantidad_total
FROM pedidos
WHERE cliente_id IS NOT NULL
GROUP BY cliente_id
)
SELECT * FROM cantidad_clientes
"""
pd.read_sql_query(query, conn)

,cliente_id,cantidad_total
0,1,3
1,2,2
2,3,2
3,4,1
4,5,3


In [ ]:
query = """
WITH cantidad_clientes AS (
    SELECT 
        cliente_id, 
        SUM(cantidad) as cantidad_total
    FROM pedidos
    WHERE cliente_id IS NOT NULL
    GROUP BY cliente_id
)
SELECT * FROM cantidad_clientes
"""
pd.read_sql_query(query, conn)